In [111]:
import DCCRN
import torch
from asteroid.data import DNSDataset
import os
import time
import torch
import shutil
import numpy as np

import asteroid
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import config as cfg
from tools_for_model import near_avg_index, max_index, min_index, Bar, cal_pesq, cal_stoi

import inspect


In [112]:
model = DCCRN.DCCRN()
#data_set =  DNSDataset("/root/NTH_student/Speech_Enhancement_new/asteroid/egs/dns_challenge/baseline/data")

In [113]:
# tạo sample để input mode. cái này random đại thui
input_size = model.stft.weight.size(1)
# tạo cái batch có n= 5 đi
input = torch.randn(8, input_size)

In [114]:
input_size

1

In [115]:
# Tạo list chứa các feature map do encoder output ra 
# dùng list vì lát nữa feature map của lstm với decoder bỏ vô đây luôn. ở đây demo cái encoder thui nha. 
feature_maps = []
# Tạo hàm hook, hàm này sẽ thực thi khi encoder thực hiện forward.
# output của encoder sẽ được append vào trong encoder_feature_map(một biến toàn cục)
def encoder_hook(module, input, output):
    feature_maps.append(output)
# đăng kí forward hook. kiểu "ê khi nào encoder thực hiện foward xong thì chạy hàm encoder_hook nha"
model.encoder[-2].register_forward_hook(encoder_hook)

In [116]:
def decoder_hook(module, input, output):
    feature_maps.append(output)

model.decoder[-2].register_forward_hook(decoder_hook)

In [117]:
def enhance_hook(module, input, output):
    feature_maps.append(output)

model.enhance.register_forward_hook(enhance_hook)

In [120]:
def out_wav(module, input, output):
    feature_maps.append(output)

model.out_wav.register_forward_hook(out_wav)

AttributeError: 'DCCRN' object has no attribute 'out_wav'

In [ ]:
try :
    model(input)
    print('au de')
except:
    print('failed roihuhu')

au de


In [ ]:
print(len(feature_maps))

3


In [ ]:
feature_maps[0].shape

torch.Size([8, 256, 8, 3])

In [ ]:
feature_maps[1][0]

tensor([[[ 0.0210,  0.0011, -0.0130,  ..., -0.0104,  0.0026, -0.0181],
         [ 0.0185,  0.0034, -0.0134,  ..., -0.0067, -0.0068, -0.0174],
         [ 0.0118,  0.0084, -0.0180,  ..., -0.0016, -0.0131, -0.0063],
         ...,
         [ 0.0096,  0.0090, -0.0129,  ..., -0.0072, -0.0112,  0.0024],
         [ 0.0257, -0.0032, -0.0075,  ..., -0.0106,  0.0078, -0.0156],
         [ 0.0164,  0.0030, -0.0142,  ..., -0.0053, -0.0102, -0.0133]],

        [[ 0.0356, -0.0248, -0.0400,  ...,  0.0074,  0.0056, -0.0162],
         [ 0.0252, -0.0039, -0.0445,  ..., -0.0091, -0.0129, -0.0178],
         [ 0.0006,  0.0093, -0.0368,  ..., -0.0084, -0.0251, -0.0180],
         ...,
         [-0.0109,  0.0137, -0.0467,  ..., -0.0061, -0.0117,  0.0028],
         [ 0.0454, -0.0272, -0.0232,  ...,  0.0190,  0.0057, -0.0033],
         [ 0.0246, -0.0021, -0.0432,  ..., -0.0124, -0.0261, -0.0261]],

        [[ 0.0154, -0.0318, -0.1036,  ..., -0.0129,  0.0323, -0.0376],
         [ 0.0293, -0.0277, -0.0863,  ..., -0

In [ ]:
feature_maps[1][1]

(tensor([[[ 1.0186e-01, -1.3451e-01,  4.6279e-02,  ..., -1.2498e-02,
            1.0699e-01, -2.2515e-01],
          [ 2.1652e-01, -1.4936e-01,  9.5088e-02,  ...,  6.2211e-02,
            2.1998e-01, -1.2541e-01],
          [ 3.8444e-02, -4.8844e-02, -1.7234e-01,  ...,  1.5596e-01,
            1.4748e-01, -6.5511e-02],
          ...,
          [ 6.6792e-03,  1.7209e-01,  2.0051e-01,  ...,  1.7541e-01,
            1.8299e-02, -1.8692e-01],
          [ 4.4413e-02, -2.0412e-01,  7.8305e-03,  ..., -7.5992e-02,
            3.8010e-02, -4.0508e-01],
          [ 1.6764e-01, -1.9624e-01,  7.6082e-02,  ...,  9.9756e-02,
            2.2322e-01, -1.1944e-01]],
 
         [[ 1.5362e-02, -3.1764e-02, -1.0362e-01,  ..., -1.2880e-02,
            3.2304e-02, -3.7595e-02],
          [ 2.9338e-02, -2.7713e-02, -8.6324e-02,  ..., -2.1035e-02,
           -9.2408e-03, -1.2835e-02],
          [ 1.3758e-02, -7.1822e-04, -2.9166e-02,  ..., -1.0332e-02,
           -9.3646e-03, -1.3909e-02],
          ...,
    

In [ ]:
feature_maps[2].shape

torch.Size([8, 16, 128, 4])

In [ ]:
feature_maps.clear()

In [ ]:
l = [[1,2,3],[1,2,3]]
s = [4,5,6]

l + s

[[1, 2, 3], [1, 2, 3], 4, 5, 6]